# Quickstart for Billiards-v0
Sam Greydanus | 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from moviepy.editor import ImageSequenceClip

import sys ; sys.path.append('..')

from billiards.utils import ObjectView, to_pickle, from_pickle
from billiards.simulate import init_balls, simulate_balls, Billiards
from billiards.render import render_masks, project_to_rgb
from billiards.dataset import load_dataset

## Setup global variables
We'll put them all in one object for organizational purposes

In [2]:
def get_args(as_dict=False):
    arg_dict = {'num_samples': 10000,
                'train_split': 0.9,
                'time_steps': 45,
                'num_balls': 2,
                'r': 1e-1,
                'dt': 1e-2,
                'seed': 0,
                'make_1d': False,
                'verbose': True,
                'side': 32,  # side lenth, in pixels
                'use_pixels': False}
    return arg_dict if as_dict else ObjectView(arg_dict)

args = get_args()

## Simulate
Simulate one trajectory and visualize it as a movie.

In [3]:
# init_state has shape [balls, x_y_vx_vy]
init_state = init_balls(args.r, args.num_balls, args.make_1d, normalize_v=False)

# simulate the system forward in time
# trajectory has shape [time, balls, x_y_vx_vy]
TIMESTEPS = 225
trajectory = simulate_balls(args.r, args.dt, TIMESTEPS, args.num_balls, init_state,
                        args.make_1d, normalize_v=False, verbose=False)

# convert the x,y coordinates into images with rgb values
# masks has shape [time, num_balls, x, y]
masks = render_masks(trajectory, r=args.r, side=32*9) # "side" determines the pixel resolution of the frames

# render the sequence of frames as a movie
# frameseq has shape [time, x, y, rgb]
upsample = lambda x: x.repeat(9, axis=0).repeat(9, axis=1)
frames = [m for m in project_to_rgb(masks.transpose(0,2,3,1))] # w/lower dpi, use "[upsample(m) for m ...]"

# turn the frames into a video
ImageSequenceClip(frames, fps=70).ipython_display()

t:  14%|█▍        | 32/225 [00:00<00:00, 315.88it/s, now=None]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


##  RL Environment API
It's just a simple Python class that wraps the Billiards simulation. You can either get pixels or coordinates as observations. Actions are two dimensional and correspond to "horizontal force" and "vertical force" applied to the purple ball at a given timestep.

In [4]:
np.random.seed(5)
env = Billiards(args, use_pixels=True)  # use pixel observations
env.state[:,2:] = 0  # make both balls start out motionless (state's axes are ["balls","x,y,vx,vy"])
all_obs = []
for timestep in range(55):
  if timestep==10:
    action = np.asarray([-.23,-.75])  # at time t=10, apply a force of 1 in the "up" direction
  else:
    action = None
  obs, reward, done, info = env.step(action)
  all_obs.append(obs)
    
all_obs = np.stack(all_obs)
print(all_obs.shape)

frameseq = [f.repeat(9, axis=0).repeat(9, axis=1) for f in all_obs]
frameseq = frameseq + [frameseq[-1]]*10
ImageSequenceClip(frameseq, fps=70./10).ipython_display()

(55, 32, 32, 3)
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


## Dataset API
For the purposes of studying model-based RL, in particular, to investigate the use of learned physics models for long-horizon planning, it is useful to generate a series of these videos and save them as a dataset. Here's how to do it:

In [5]:
args = get_args()
args.num_samples = 100 # size of our dataset
args.use_pixels = True # whether to save pixel data as part of our dataset (otherwise, it just saves coords)

# try loading the dataset; if it can't be found, then generate it from scratch
t0 = time.time()
path = './billiards.pkl'
dataset = load_dataset(args, path=path, regenerate=True)
print("Made/loaded the dataset; it took {:.2e}s\n".format(time.time()-t0))

# visualize the 60th trajectory in the dataset
dataset['x'].shape
frameseq = [f.repeat(9, axis=0).repeat(9, axis=1) for f in dataset['x'][:,60]]
ImageSequenceClip(frameseq, fps=70./10).ipython_display()

Did or could not load data from ./billiards.pkl. Rebuilding dataset...
When Sam profiled this code, it took 0.15 sec/trajectory.
	-> Expect it to take ~25 mins to generate 10k samples.
dataset 100.000% built

Made/loaded the dataset; it took 1.89e+01s

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


### Notice that the second time we call the load_dataset function it is much faster

In [6]:
# try loading the dataset; if it can't be found, then generate it from scratch
t0 = time.time()
path = './billiards.pkl'
dataset = load_dataset(args, path=path)
print("Made/loaded the dataset; it took {:.2e}s\n".format(time.time()-t0))

Successfully loaded data from ./billiards.pkl
Made/loaded the dataset; it took 2.92e-02s



## Make a coordinate dataset

In [7]:
args = get_args()
args.num_samples = 100 # size of our dataset
args.use_pixels = False # whether to save pixel data as part of our dataset (otherwise, it just saves coords)

# try loading the dataset; if it can't be found, then generate it from scratch
t0 = time.time()
path = './billiards.pkl'
dataset = load_dataset(args, path=path, regenerate=True)
print("Made/loaded the dataset; it took {:.2e}s\n".format(time.time()-t0))

Did or could not load data from ./billiards.pkl. Rebuilding dataset...
dataset 100.000% builtMade/loaded the dataset; it took 2.56e+00s



In [8]:
dataset['x'][:20,0,:].round(1)

array([[ 0.9,  0.7,  0.4,  0.7,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.7,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.8,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.8, -0.7,  0.8],
       [ 0.9,  0.7,  0.3,  0.8,  0. ,  0. ],
       [ 0.9,  0.7,  0.3,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.9,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.9,  0. ,  0. ],
       [ 0.9,  0.6,  0.1,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.1,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.7,  0. ,  0. ],
       [ 0.9,  0.6,  0.3,  0.7,  0. ,  0. ],
       [ 0.8,  0.5,  0.3,  0.6,  0. ,  0. ],
       [ 0.8,  0.5,  0.4,  0.6,  0. ,  0. ],
       [ 0.8,  0.5,  0.5,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.5,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.4,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.4,  0. ,  0. ]])

In [9]:
dataset['y'][:20,0,:].round(1)

array([[ 0.9,  0.7,  0.4,  0.7,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.7,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.8,  0. ,  0. ],
       [ 0.9,  0.7,  0.4,  0.8, -0.7,  0.8],
       [ 0.9,  0.7,  0.3,  0.8,  0. ,  0. ],
       [ 0.9,  0.7,  0.3,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.9,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.9,  0. ,  0. ],
       [ 0.9,  0.6,  0.1,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.1,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.8,  0. ,  0. ],
       [ 0.9,  0.6,  0.2,  0.7,  0. ,  0. ],
       [ 0.9,  0.6,  0.3,  0.7,  0. ,  0. ],
       [ 0.8,  0.5,  0.3,  0.6,  0. ,  0. ],
       [ 0.8,  0.5,  0.4,  0.6,  0. ,  0. ],
       [ 0.8,  0.5,  0.5,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.5,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.5,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.4,  0. ,  0. ],
       [ 0.8,  0.5,  0.6,  0.4,  0. ,  0. ]])